In [1]:
from bokeh.plotting import figure
from bokeh.resources import CDN
from bokeh.embed import file_html

plot = figure()
plot.circle([1,2,3], [3,4,5])

html = file_html(plot, CDN, "my plot")

In [2]:
with open('html/plot.html', 'w') as f:
    f.write(html)

In [3]:
# scatter.py

from bokeh.plotting import figure
from bokeh.models import Range1d
from bokeh.embed import components

# create some data
x1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
y1 = [0, 8, 2, 4, 6, 9, 5, 6, 25, 28, 4, 7]
x2 = [2, 5, 7, 15, 18, 19, 25, 28, 9, 10, 4]
y2 = [2, 4, 6, 9, 15, 18, 0, 8, 2, 25, 28]
x3 = [0, 1, 0, 8, 2, 4, 6, 9, 7, 8, 9]
y3 = [0, 8, 4, 6, 9, 15, 18, 19, 19, 25, 28]

# select the tools we want
TOOLS="pan,wheel_zoom,box_zoom,reset,save"

# the red and blue graphs will share this data range
xr1 = Range1d(start=0, end=30)
yr1 = Range1d(start=0, end=30)

# only the green will use this data range
xr2 = Range1d(start=0, end=30)
yr2 = Range1d(start=0, end=30)

# build our figures
p1 = figure(x_range=xr1, y_range=yr1, tools=TOOLS, plot_width=300, plot_height=300)
p1.scatter(x1, y1, size=12, color="red", alpha=0.5)

p2 = figure(x_range=xr1, y_range=yr1, tools=TOOLS, plot_width=300, plot_height=300)
p2.scatter(x2, y2, size=12, color="blue", alpha=0.5)

p3 = figure(x_range=xr2, y_range=yr2, tools=TOOLS, plot_width=300, plot_height=300)
p3.scatter(x3, y3, size=12, color="green", alpha=0.5)

# plots can be a single Bokeh Model, a list/tuple, or even a dictionary
plots = {'Red': p1, 'Blue': p2, 'Green': p3}

script, div = components(plots)

In [4]:
from jinja2 import Template
t = Template("""<!DOCTYPE html>
             <html lang="en">
    <head>
        <meta charset="utf-8">
        <title>Bokeh Scatter Plots</title>

        <link rel="stylesheet" href="https://cdn.bokeh.org/bokeh/release/bokeh-1.3.4.min.css" type="text/css" />
        <script type="text/javascript" src="https://cdn.bokeh.org/bokeh/release/bokeh-1.3.4.min.js"></script>

        {{ script }}

    </head>
    <body>
        {{ div }}
    </body>
</html>""")


In [5]:
with open('html/plot.html', 'w') as f:
    f.write(t.render(script=script, div=div))

In [6]:
!cat html/plot.html

<!DOCTYPE html>
             <html lang="en">
    <head>
        <meta charset="utf-8">
        <title>Bokeh Scatter Plots</title>

        <link rel="stylesheet" href="https://cdn.bokeh.org/bokeh/release/bokeh-1.3.4.min.css" type="text/css" />
        <script type="text/javascript" src="https://cdn.bokeh.org/bokeh/release/bokeh-1.3.4.min.js"></script>

        
<script type="text/javascript">
  (function() {
    var fn = function() {
      Bokeh.safely(function() {
        (function(root) {
          function embed_document(root) {
            
          var docs_json = '{"60b9f019-7ff7-478b-a405-b5bc50cd0b62":{"roots":{"references":[{"attributes":{},"id":"1203","type":"BasicTicker"},{"attributes":{"source":{"id":"1152","type":"ColumnDataSource"}},"id":"1156","type":"CDSView"},{"attributes":{"dimension":1,"ticker":{"id":"1203","type":"BasicTicker"}},"id":"1206","type":"Grid"},{"attributes":{"fill_alpha":{"value":0.5},"fill_color":{"value":"blue"},"line_alpha":{"value":0.5},"line_color